In [ ]:
%matplotlib widget
from sympy import *
init_printing(use_latex=True)

# 15 - The Equation class

In [ ]:
from algebra_with_sympy import *
# by default, when an equation is shown on the screen, it also shows
# a label with a unique number. Usually, I hide them:
algwsym_config.output.label = False
# by default, the algebra_with_sympy's solve returns objects of type
# FiniteSet. I don't like that behavior: I prefer lists of solutions.
algwsym_config.output.solve_to_list = True

In [ ]:
a, b, c, x = symbols("a, b, c, x")
eq1 = Equation(a + b, c * x)
eq2 = Equation(x**2 - 4 * x, a + b)
display(eq1, eq2)

In [ ]:
display(eq1.lhs, eq1.rhs, eq1.reversed)

In [ ]:
res = solve(eq1, x)
res

In [ ]:
eq1 + 2

In [ ]:
eq1 / eq2

In [ ]:
eq1.applyrhs(lambda t: t**2)

In [ ]:
eq1.subs(c, 2)

In [ ]:
eq1.subs(eq2.reversed)

## 15.1 - Electric Circuit

In [ ]:
L, R, C1, C2, t = symbols("L, R, C1, C2, t")
iC1, iC2, VL, Vi = symbols("i_{C_{1}}, i_{C_{2}}, V_L, V_i")
VC1, VC2, iL = [f(t) for f in symbols("V_{C_{1}}, V_{C_{2}}, i_L", cls=Function)]

In [ ]:
iC2_eq = Equation(iC2, (VL - VC2) / R)
iC1_eq = Equation(iC1, iL + iC2)
VL_eq = Equation(VL, Vi - VC1)
display(iC1_eq, iC2_eq, VL_eq)

In [ ]:
e1 = Equation(C1 * VC1.diff(t), iC1)
e2 = Equation(L * iL.diff(t), VL)
e3 = Equation(C2 * VC2.diff(t), iC2)
display(e1, e2, e3)

In [ ]:
e1 = (e1.subs(iC1_eq).subs(iC2_eq).subs(VL_eq) / C1).expand()
e2 = (e2.subs(VL_eq) / L).expand()
e3 = (e3.subs(iC2_eq).subs(VL_eq) / C2).expand()
display(e1, e2, e3)

In [ ]:
A, b = linear_eq_to_matrix([e.rhs for e in [e1, e2, e3]], [VC1, VC2, iL])
A, b

In [ ]:
final = Equation(
    Matrix([e.lhs for e in [e1, e2, e3]]),
    MatMul(A, b, evaluate=False))
final

## 15.2 - Temperature Distribution

In [ ]:
t, k, x, m, Cp, rho, alpha = symbols(r"t, k, x, m, C_p, rho, alpha", positive=True)
Dt, Dx, Dy, Dz, DV = [symbols(r"{\Delta}" + s, positive=True) for s in "t,x,y,z,V".split(",")]
Q, q, P, T = [e(t, x) for e in symbols("Q, q, P, T", cls=Function)]

In [ ]:
Mass = Equation(rho, m / DV)
Veq = Equation(DV, Dx * Dy * Dz)
Qeq = Equation(Q, m * Cp * T)
fourier = Equation(q, -k * T.diff(x))
td = Equation(alpha, k / (Cp * rho))
display(Mass, Veq, Qeq, fourier, td)

In [ ]:
# change in heat content
hc_change = Q - Q.subs(t, t + Dt)
# heat in: Dy * Dz is the surface area through which q flows
hi = q * Dt * Dy * Dz
# heat out: Dy * Dz is the surface area through which q flows
ho = q.subs(x, x + Dx) * Dt * Dy * Dz
# heat generated
hg = P * Dt * Dx * Dy * Dz
# heat balance equation
hb = Equation(hc_change, hi - ho + hg)
hb

In [ ]:
Veq = Equation(DV, Dx * Dy * Dz)
hb = hb.applyrhs(lambda e: ((e * Dx).expand().subs(*Veq.reversed.args) / Dx)).expand()
hb

In [ ]:
hb = (hb / Dt).expand()
hb

In [ ]:
hb = hb.applylhs(lambda e: Limit(e.collect(1 / Dt), Dt, 0))
hb

In [ ]:
hb = hb.applylhs(lambda e: e.subs(e, Q.diff(t)))
hb

In [ ]:
qxdx = Equation(q.subs(x, x + Dx), q.subs(x, x + Dx).series(Dx, 0, 2).removeO().doit())
qxdx

In [ ]:
hb = hb.subs(qxdx).expand()
hb

In [ ]:
hb = hb.subs(fourier).subs(Qeq).doit()
hb

In [ ]:
hb = (hb / DV / k).expand().subs(Mass.reversed)
hb

In [ ]:
hb = hb.subs(1 / td.reversed)
hb